In [144]:
import os
import re
import geopy as gp
import numpy as np
import pandas as pd
import altair as alt
import geopandas as gpd
#from uszipcode import SearchEngine
from Modules import collision_preprocessing as cp

In [145]:
dir = './Data'
temp_pre = './Data/tmp_pre'
colission_exists = False

## Weather dataset preprocessing

In the process of downloading the data, we selected all the attributes available. Now, we will explore the data and select the ones that are useful for our purpose.

In [146]:
weather = pd.read_csv(f'{dir}/weather.csv')
weather.shape

D:\USUARIOS\Enric-UPC\AppData\Local\Temp\ipykernel_2408\941678255.py:1: DtypeWarning: Columns (7,9,13,17,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv(f'{dir}/weather.csv')


(90591, 66)

### Data exploration

In [147]:
weather.shape

(90591, 66)

Since we are only interested in the rows where the date is inside the timeranges of 01/06/2018 - 31/09/2018 and 01/06/2020 - 31/09/2020, we will filter the data to only include those rows.

In [148]:
weather = weather[((weather['DATE'] >= '2018-06-01') & (weather['DATE'] <= '2018-09-30')) | ((weather['DATE'] >= '2020-06-01') & (weather['DATE'] <= '2020-09-30'))]

weather.to_csv(f'{dir}/weather_2018-2020.csv', index=False) # sobre el mismo archivo

In [149]:
weather.shape

(20536, 66)

In [150]:
weather.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'AWND_ATTRIBUTES', 'DAPR', 'DAPR_ATTRIBUTES', 'DASF', 'DASF_ATTRIBUTES',
       'MDPR', 'MDPR_ATTRIBUTES', 'MDSF', 'MDSF_ATTRIBUTES', 'PGTM',
       'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'PSUN', 'PSUN_ATTRIBUTES',
       'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG',
       'TAVG_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES',
       'TOBS', 'TOBS_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WDF2',
       'WDF2_ATTRIBUTES', 'WDF5', 'WDF5_ATTRIBUTES', 'WESD', 'WESD_ATTRIBUTES',
       'WESF', 'WESF_ATTRIBUTES', 'WSF2', 'WSF2_ATTRIBUTES', 'WSF5',
       'WSF5_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT02', 'WT02_ATTRIBUTES',
       'WT03', 'WT03_ATTRIBUTES', 'WT04', 'WT04_ATTRIBUTES', 'WT05',
       'WT05_ATTRIBUTES', 'WT06', 'WT06_ATTRIBUTES', 'WT08', 'WT08_ATTRIBUTES',
       'WT09', 'WT09_ATTRIBUTES', 'WT11', 'WT11_ATTRIBUTES'],
      dtype='object')

By looking at the documentation of the weather datatset and the attributes present, each row represents some selected observations (values) available for a given **STATION** and **DATE**. Neither the **STATION** nor the **DATE** are unique.

In [151]:
len(weather['STATION'].unique()), len(weather['DATE'].unique())

(129, 244)

Apart from the first 6 columns listed avobe, the rest of the attributes correspond to optional flags and their respective attributes (definded by the weather documentation *Note: The 4 flags listed ... are optional on the Custom GHCN-Daily ASCII Form*) and therefore can contain several null values. That is the reason for the large quantity of sparse attributes detected. We will explore the data to see which attributes are useful for our purpose.

All these atributes correspond to the Table 4. Some of the most relevant to understand the weather conditons are:

AWND, DAPR, DASF, MDPR, MDSF, PGTM, PRCP, PSUN, SNOW, SNWD, TAVG, TMAX, TMIN, TOBS, TSUN, WDF2, WDF5, WESD, WESF, WSF2, WSF5, WT01, WT02, WT03, WT04, WT05, WT06, WT08, WT09, WT11

### Data selection

First of all, we will start by joining **LATITUDE** and **LONGITUDE** in a **LOCATION** attribute following the format of the *collision* dataset. Since both attributes ahave no missing values, the resulting column will not have any missing values.

In [152]:
weather[(weather['LATITUDE'].notnull()) & (weather['LONGITUDE'].notnull())].shape


(20536, 66)

In [153]:
weather['LOCATION'] = '(' + weather['LATITUDE'].astype(str) + ', ' + weather['LONGITUDE'].astype(str) + ')'

We will delete the **ELEVATION** column since it can't be related in any way with the previous dataset and the information it gives is not useful for our visualization purpose.

In [154]:
weather.drop(columns=['LATITUDE', 'LONGITUDE', 'ELEVATION'], inplace=True)

Since we have *LOCATION*, we don't need *STATION* code or *NAME*.

-- yo no los quitaria aun, porque si queremos hacer un mapa de estaciones, necesitamos el codigo de estacion y el nombre de la estacion. Si no, no sabemos a que estacion corresponde cada punto.

In [155]:
# weather.drop(columns=['STATION', 'NAME'], inplace=True)

Now, with regard to the other columns, let's follow the proposed startegy. We are interested in **LOCATION** so we can know about the weather condtitions of the zone where the collision was produced. For the weather attributes, following the documentation of the weather dataset, provided by *NOAA* and available in the ``Documentation`` directory, we are interested in:

- *PRCP* : Precipitation (mm)
- *SNOW* : Snowfall (mm)
- *SNWD* : Snow depth (mm)
- *TMAX* : Maximum temperature (Celsius)
- *TMIN* : Minimum temperature (Celsius)
- *TOBS* : Temperature at the time of observation
- *AWND* : Average daily wind speed (meters per second)

Since we are analyzing the summer periods of 2018 and 2020, we could probably drop the *SNOW* attribute since probably all values will be 0. However, let's check it out before deleting the column.

In [156]:
weather['SNOW'].value_counts()

SNOW
0.0    9222
Name: count, dtype: int64

As we supposed, the not null values of *SNOW* are all 0, and we can assume that the null ones would be 0 too. So we can safely delete this column as it makes no sense to analize snow during summer.

In [157]:
weather.drop(columns=['SNOW'], inplace=True)

In [158]:
weather['WT09_ATTRIBUTES'].unique()

array([nan], dtype=object)

In [160]:
select_col = ['DATE',
              'PRCP',
              'SNWD',
              'TMAX',
              'TMIN',
              'TOBS',
              'AWND',
              'WT01',
              'WT02',
              'WT03',
              'WT04',
              'WT05',
              'WT06',
              'WT08',
              'WT09',
              'WT11',
              'LOCATION']

weather = weather[select_col]

# change 'LOCATION' column to the second column
cols = list(weather.columns)
cols = [cols[-1]] + cols[:-1]
weather = weather[cols]

In [161]:
weather.columns

Index(['LOCATION', 'DATE', 'PRCP', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'AWND',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11'],
      dtype='object')

In [162]:
weather[weather['WT01'].isnull() & weather['WT02'].isnull() & weather['WT03'].isnull() & weather['WT04'].isnull() & weather['WT05'].isnull() & weather['WT06'].isnull() & weather['WT08'].isnull() & weather['WT09'].isnull() & weather['WT11'].isnull()].shape

(19894, 17)

In [163]:
# todas las filas nulas pueden ser que el dia haya sido bonito
# WT** son binarios, si es nulo es que no paso nada
# si hay alguno de los WT que no es nulo, entonces el dia no fue bonito

In [164]:
weather['WT01'].unique(), weather['WT02'].unique(), weather['WT03'].unique(), weather['WT04'].unique(), weather['WT05'].unique(), weather['WT06'].unique(), weather['WT08'].unique(), weather['WT09'].unique(), weather['WT11'].unique()

(array([nan,  1.]),
 array([nan,  1.]),
 array([nan,  1.]),
 array([nan]),
 array([nan,  1.]),
 array([nan]),
 array([nan,  1.]),
 array([nan]),
 array([nan,  1.]))

In [165]:
weather.columns

Index(['LOCATION', 'DATE', 'PRCP', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'AWND',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11'],
      dtype='object')

In [166]:
wt_columns = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11']

weather['ADVERSE_CONDITIONS'] = weather[wt_columns].isnull().all(axis=1).astype(int)
weather['ADVERSE_CONDITIONS'] = 1 - weather['ADVERSE_CONDITIONS']

weather['ADVERSE_CONDITIONS'].value_counts()

ADVERSE_CONDITIONS
0    19894
1      642
Name: count, dtype: int64

In [167]:
weather.drop(columns=wt_columns, inplace=True)
weather.head()

,LOCATION,DATE,PRCP,SNWD,TMAX,TMIN,TOBS,AWND,ADVERSE_CONDITIONS
151,"(40.89174, -74.39635)",2018-06-01,1.8,0.0,20.6,16.7,18.9,NaN,0
152,"(40.89174, -74.39635)",2018-06-02,5.3,0.0,28.9,18.9,21.7,NaN,0
153,"(40.89174, -74.39635)",2018-06-03,14.0,0.0,29.4,14.4,15.0,NaN,0
154,"(40.89174, -74.39635)",2018-06-04,16.8,0.0,18.9,10.6,11.1,NaN,0
155,"(40.89174, -74.39635)",2018-06-05,0.0,0.0,23.3,11.1,12.2,NaN,0


### Missing values

In [168]:
weather.isnull().sum()

LOCATION                  0
DATE                      0
PRCP                    297
SNWD                  16813
TMAX                  17221
TMIN                  17233
TOBS                  19244
AWND                  18608
ADVERSE_CONDITIONS        0
dtype: int64

 For *PRCP*, we will assume that the null values are 0, that is if there's no record of precipitation (the value is null), it's because there were no precipitations. We make this assumption because the number of null values represents around 10% of the dataset so it's not such a risk to make this assumption (and even less risk taking into account that the data is from Summer, when it's less likely to rain).

In [169]:
weather['PRCP'].fillna(0.0, inplace=True)

In [170]:
weather.drop(columns=['SNWD'], inplace=True)

Regarding temperature, the great number of missing values is a problem. We will impute the missing values with the avergae temperature per day of each attribute (*TMAX*, *TMIN*, *TOBS*).

In [171]:
weather['TMAX'] = weather.groupby('DATE')['TMAX'].transform(lambda x: x.fillna(x.mean()))
weather['TMIN'] = weather.groupby('DATE')['TMIN'].transform(lambda x: x.fillna(x.mean()))
weather['TOBS'] = weather.groupby('DATE')['TOBS'].transform(lambda x: x.fillna(x.mean()))


In [172]:
weather.isnull().sum()

LOCATION                  0
DATE                      0
PRCP                      0
TMAX                      0
TMIN                      0
TOBS                      0
AWND                  18608
ADVERSE_CONDITIONS        0
dtype: int64

We will apply the same strategy on AWND to get rid of the missing values.

In [173]:
weather['AWND'] = weather.groupby('DATE')['AWND'].transform(lambda x: x.fillna(x.mean()))

In [174]:
weather.isnull().sum()

LOCATION              0
DATE                  0
PRCP                  0
TMAX                  0
TMIN                  0
TOBS                  0
AWND                  0
ADVERSE_CONDITIONS    0
dtype: int64

In [175]:
# Podemos categorizar AWND con la Beaufort scale (https://en.wikipedia.org/wiki/Beaufort_scale)

# Podemos categorizar PRCP con respecto a la World Meteorological Organization, 2018(https://www.researchgate.net/figure/Rain-intensity-classifications-according-to-the-World-Meteorological-Organization-2018_tbl1_353769617)
